<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/6.Gestion_de_datos/Agregaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Agregación de datos y operaciones de grupo](#scrollTo=BFynk27lvXm6)

>>[Actividades que veremos en este apartado](#scrollTo=EMGwZJuAv306)

>>>[Mecánica del GroupBy](#scrollTo=F1ju7vADwWy1)

>>>[Seleccionando una columna o subset de columnas](#scrollTo=_SAtsTx-1xjS)

>>>[Agrupando con dicts y series](#scrollTo=SFpRZZb23nOj)

>>>[Agrupación con funciones](#scrollTo=hNvhRjQg5IhT)

>>>[Data Aggregation](#scrollTo=tCCi7PrE5ecH)

>>>[Aplicación de columna inteligente y de funciones múltiples](#scrollTo=rbeK_M1blP_0)

>>[Aplicar: general dividir-aplicar-combinar](#scrollTo=K8sgtonjqmcI)

>>>[Análisis de cuantiles y buckets](#scrollTo=mFfqx7P0sbeq)

>>>[Rellenar valores perdidos con valores específicos de grupo](#scrollTo=tvKXddD9tV0C)

>>[Muestreo aleatorio y permutación](#scrollTo=vTsm2BuHv_SC)

>>>[Promedio ponderado grupal y correlación](#scrollTo=eX6B4Plhxr8P)

>>[Pivot Tables y tabulación cruzada](#scrollTo=3e5jR8qh0UMW)

>>>[Tabulaciones cruzadas (crosstab)](#scrollTo=gprrjJ0m12nf)



# Agregación de datos y operaciones de grupo

La categorización de un conjunto de datos y la aplicación de una función a cada grupo, ya sea una agregación o transformación, es un componente crítico del trabajo de análisis de datos. Después de cargar, fusionar y preparar un conjunto de datos, es posible que debas calcular estadísticas de grupo o posiblemente tablas dinámicas para fines de informes o visualización. Pandas proporciona una interfaz de grupo flexible, que te permite cortar, y resumir conjuntos de datos de forma natural.

Como verás, con la expresividad de Python y pandas, podemos realizar operaciones grupales bastante complejas utilizando cualquier función que acepte un objeto pandas o una matriz NumPy. 

## Actividades que veremos en este apartado

* Dividir un objeto pandas en pedazos usando una o más claves (en forma de funciones, matrices o nombres de columna de DataFrame)

* Calcular estadísticas de resumen de grupo, como conteo, media o desviación estándar, o una función definida por el usuario

* Aplique transformaciones dentro del grupo u otras manipulaciones, como normalización, regresión lineal, clasificación o selección de subconjuntos

* Calcular tablas dinámicas y tabulaciones cruzadas

* Realizar análisis cuantiles y otros análisis de grupos estadísticos.

### Mecánica del GroupBy

Existe un término conocido entre los analistas que describe operaciones de grupo, *split-apply-combine* 
En la primera parte de este proceso dividimos (split) dataframes o series en grupos basados en una o más keys. Una vez realizado la división, realizamos la función *apply* a cada grupo, produciendo un nuevo valor. 
Finalmente, tomamos el resultado de esas operaciones y las combinamos en un objeto.

![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/split-apply-combine.png)

*source: Python for Data Analysis, 2nd Edition*



In [0]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

Supongamos que deseas calcular la media *(mean())* de la columna data1 usando las etiquetas de key1

In [0]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [0]:
grouped.mean()

Aquí agrupamos los datos usando dos claves, y la Serie resultante ahora tiene un índice jerárquico.

In [0]:
media = df['data1'].groupby([df['key1'], df['key2']]).mean()
media

In [0]:
media.unstack()

En el siguiente ejemplo todo el grupo de keys son series

In [0]:
prov = np.array(['Buenos Aires', 'Buenos Aires', 'Córdoba', 'Córdoba', 'Tucumán'])
anios = np.array([2005, 2005, 2005, 2006, 2006])

In [0]:
df['data1']

In [0]:
df['data1'].groupby([prov, anios]).mean()

### Seleccionando una columna o subset de columnas

La indexación de un objeto **GroupBy** creado a partir de un DataFrame con un nombre de columna o matriz de nombres de columna, genera un subconjunto de columnas para la agregación.

In [0]:
df.groupby('key1')['data1'] # Equivalente a df['data1'].groupby(df['key1'])
df.groupby('key1')['data2'] # Equivalente a df[['data2']].groupby(df['key1'])

Especialmente para grandes conjuntos de datos, puede ser conveniente agregar solo unas pocas columnas. Por ejemplo, en el conjunto de datos anterior, para calcular medios solo para la columna data2 y obtener el resultado como un DataFrame, podríamos escribir:

In [0]:
df.groupby(['key1', 'key2'])[['data2']].mean()

El objeto devuelto por esta operación de indexación es un DataFrame agrupado. Sera una lista o matriz o una Serie agrupada si solo se pasa un solo nombre de columna como escalar

In [0]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.mean()

### Agrupando con dicts y series

Puede que necesites agrupar información existente en algo diferente a un arreglo. Consideremos el siguiente Dataframe:


In [0]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

In [0]:
people.iloc[2:3, [1, 2]] = np.nan # Agrega un par de NaN
people

Supongamos que tenemos una lista de columnas que corresponden a ese dataframe y queremos realizar una operacion **sum** entre las columnas por grupo

In [0]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

Ahora podemos construir un arreglo a partir del dict y se lo pasamos a la operación **groupby**, pero en cambio le pasamos directamente el dict como key.


In [0]:
by_column=people.groupby(mapping, axis=1)
by_column.sum()

### Agrupación con funciones
El uso de las funciones de Python es una forma más genérica de definir un mapeo de grupo en comparación con un dict o Series. Cualquier función que se pase como clave de grupo se llamará una vez por valor de índice, y los valores de retorno se utilizarán como nombres de grupo. Más concretamente, consideremos el DataFrame de ejemplo de la sección anterior, que tiene los nombres de las personas como valores de índice. Supongamos que deseas agrupar por la longitud de los nombres; Si bien podrías calcular una matriz de longitudes de cadena, es más simple simplemente pasar la función len:

In [0]:
people.groupby(len).sum()

### Data Aggregation
Las agregaciones se refieren a cualquier transformación de datos que produce valores escalares a partir de matrices. Los ejemplos anteriores han utilizado varios de ellos, incluidos la media, el recuento, el mínimo y la suma. 


Function name |	Description
------------- | -----------
count	| Número de valores no-NA en el grupo
sum	| Suma de valores no-NA
mean	| Media de valores no-NA 
median	| Mediana aritmética de valores no-NA
std, var	| Desviación y varianza estándar imparcial (denominador n - 1)
min, max	| Mínimo y máximo de valores no-NA
prod	| Producto de valores no-NA 
first, last	| Primer y último valores no-NA 




Puedes usar agregaciones de tu propio diseño y, además, llamar a cualquier método que también esté definido en el objeto agrupado. Por ejemplo, puedes recordar que el cuantil calcula los cuantiles de muestra de una serie o las columnas de un marco de datos.

Si bien el cuantil no se implementa explícitamente para GroupBy, es un método de la Serie y, por lo tanto, está disponible para su uso. Internamente, GroupBy corta eficientemente la serie, llama a **piece.quantile(0.9)** para cada pieza y luego ensambla esos resultados en el objeto de resultado:

In [0]:
df

In [0]:
grouped = df.groupby('key1')

In [0]:
grouped['data1'].quantile(0.9)

Puedes notar que algunos métodos como describe también funcionan, aunque no son agregaciones, estrictamente hablando

In [0]:
grouped.describe()

### Aplicación de columna inteligente y de funciones múltiples

Volvamos al conjunto de datos de propinas de ejemplos anteriores. Después de cargarlo con read_csv, agregamos una columna de porcentaje de propina tip_pct

In [0]:
prop = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/tips.csv')
prop['tip_pct'] = prop['tip'] / prop ['total_bill']


In [0]:
prop[:6]

Como hemos visto, agregar una Serie o todas las columnas de un dataframe de datos es una cuestión de utilizar el agregado con la función deseada o llamar a un método como **mean** o **std**. 
Sin embargo, es posible que desees agregar usando una función diferente dependiendo de la columna, o múltiples funciones a la vez. 

In [0]:
grouped = prop.groupby(['day', 'smoker'])

In [0]:
grouped_pct = grouped['tip_pct']

Ten en cuenta que para estadísticas descriptivas como las de la Tabla que hemos compartido al comienzo, puedes pasar el nombre de la función como una cadena, en este caso 'mean'

In [0]:
grouped_pct.agg('mean')

In [0]:
grouped_pct.agg(['min','max'])

In [0]:
def peak_to_peak(arr):      # Funcion de agregación propia 
  return arr.max() - arr.min()

Si pasas una lista de funciones o nombres de funciones, obtiene un DataFrame con nombres de columnas tomados de las funciones.

In [0]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

Aquí pasamos una lista de funciones de agregación a **agg** para evaluar de forma independiente en los grupos de datos.

No necesita aceptar los nombres que GroupBy le da a las columnas; en particular, las funciones lambda tienen el nombre '<lambda>', lo que hace que sean difíciles de identificar. Por lo tanto, si pasas una lista de tuplas (nombre, función), el primer elemento de cada tupla se usará como los nombres de columna de DataFrame.

In [0]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

Con un DataFrame tienes más opciones, ya que puedes especificar una lista de funciones para aplicar a todas las columnas o diferentes funciones por columna. Para comenzar, supongamos que deseamos calcular las mismas tres estadísticas para las columnas tip_pct y total_bill

In [0]:
functions = ['count', 'mean', 'max'] # Lista de funciones

In [0]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

'''
A las dos columnas del DF le aplicamos las tres funciones
'''
result

Ahora, supongamos que deseamos aplicar funciones potencialmente diferentes a una o más de las columnas. Para hacer esto, pasamos un dict a *agg* que contenga una asignación de nombres de columna a cualquiera de las especificaciones de funciones enumeradas hasta ahora

In [0]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

In [0]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

## Aplicar: general dividir-aplicar-combinar

El método mas general de uso de GroupBy es apply.
Como se ilustra en la Figura, apply divide el objeto que se está manipulando en piezas, invoca la función pasada en cada pieza y luego intenta concatenar las piezas juntas.
![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/split-apply-combine.png)

*source: Python for Data Analysis, 2nd Edition*

Supongamos que deseamos seleccionar los cinco valores principales de **tip_pct** por grupo. Primero, escribimos una función que seleccione las filas con los valores más grandes en una columna particular:

In [0]:
def top(df, n=5, column='tip_pct'):
  return df.sort_values(by=column)[-n:]


In [0]:
top(prop, n=6) # Llamada a la función top

Ahora, si agrupamos por fumador, por ejemplo, y aplicamos *call* con esta función, obtenemos lo siguiente:

In [0]:
prop.groupby('smoker').apply(top) # apply llama a la función top

¿Qué ha pasado aquí? La función superior se llama en cada grupo de filas desde el DF, y luego los resultados se pegan usando *pandas.concat*, etiquetando las piezas con los nombres de los grupos. Por lo tanto, el resultado tiene un índice jerárquico cuyo nivel interno contiene valores de índice del DataFrame original.

Si pasas una función a *apply* que toma otros argumentos o palabras clave, puedes pasarlos después de la función:

In [0]:
prop.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

### Análisis de cuantiles y buckets

Pandas tiene algunas herramientas, en particular *cut* y *qcut*, para dividir los datos en cubos con contenedores de tu elección o por cuantiles de muestra. La combinación de estas funciones con *groupby* hace que sea conveniente realizar análisis de buckets o cuantil en un conjunto de datos. Considere un conjunto de datos aleatorio simple y una categorización de bucket de igual longitud usando cut:

In [0]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [0]:
quartiles = pd.cut(frame.data1, 4)

In [0]:
quartiles[:10]

El objeto  devuelto por *cut* se puede pasar directamente a *groupby*. Entonces podríamos calcular un conjunto de estadísticas para la columna data2 de la siguiente manera:

In [0]:
def get_stats(group):
  return {'min': group.min(), 'max': group.max(),
          'count': group.count(), 'mean': group.mean()}

In [0]:
grouped = frame.data2.groupby(quartiles)

In [0]:
grouped.apply(get_stats) # que puedo agregar para que se vea mejor? ..un___..?

### Rellenar valores perdidos con valores específicos de grupo

Al limpiar los datos faltantes, en algunos casos reemplazarás las observaciones de datos usando *dropna*, pero en otros puedes querer completar los valores nulos (NA) usando un valor fijo o algún valor derivado de los datos. *fillna* es la herramienta adecuada para usar; por ejemplo, aquí rellenamos los valores de NA con la media:

In [0]:
s = pd.Series(np.random.randn(6))

In [0]:
s[:3] = np.nan
s

In [0]:
s.fillna(s.mean())

Supongamos que necesitas que el valor de relleno varíe según el grupo. Una forma de hacer esto es agrupar los datos y usar *apply* con una función que llame a *fillna* en cada fragmento de datos. Aquí hay algunos datos de muestra sobre los estados de EE. UU. Divididos en regiones orientales y occidentales:

In [0]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [0]:
# group_key = ['East'] * 4 + ['West'] * 4 # Notación alternativa
group_key = ['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

Ten en cuenta que la sintaxis ['Este'] * 4 produce una lista que contiene cuatro copias de los elementos en ['Este'].

In [0]:
data = pd.Series(np.random.randn(8), index=states)
data

In [0]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

In [0]:
data.groupby(group_key).mean()

In [0]:
fill_mean = lambda g: g.fillna(g.mean()) # Que hace esta funcion lambda?

In [0]:
data.groupby(group_key).apply(fill_mean)

## Muestreo aleatorio y permutación
Supongamos que deseas extraer una muestra aleatoria (con o sin reemplazo) de un gran conjunto de datos para fines de simulación o alguna otra aplicación. Hay varias formas de realizar los "sorteos"; Aquí usamos el método de muestra para Series.

In [0]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names) # Para cada letra itera por la cantidad de cartas

deck = pd.Series(card_val, index=cards)

Así que ahora tenemos una Serie de longitud 52 cuyo índice contiene nombres y valores de cartas que se usan en Blackjack y otros juegos (para simplificar las cosas, solo dejo que el as 'A' sea 1):


In [0]:
deck[:13]

In [0]:
def draw(deck, n=5):
  return deck.sample(n)

In [0]:
draw(deck)

Supongamos que quieres dos cartas al azar de cada palo. Debido a que el palo es el último personaje de cada nombre de tarjeta, podemos agruparlo en base a esto y usar apply:

In [0]:
get_suit = lambda card: card[-1] # ultima letra es el palo

In [0]:
deck.groupby(get_suit).apply(draw, n=2)

### Promedio ponderado grupal y correlación

Bajo el paradigma de *split-apply-combine* de groupby, las operaciones entre columnas en un DataFrame o dos Series, como un promedio ponderado de grupo, son posibles.
Consideremos un conjunto de datos financieros originalmente obtenido de Yahoo! Finance que contiene precios al final del día para algunas acciones y el índice S&P 500 (el símbolo SPX):

In [0]:
close_px = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/stocks.csv', 
                       parse_dates=True,index_col=0)
close_px.info()

In [0]:
close_px[-5:]

Una tarea de interés podría ser calcular un DataFrame que consta de las correlaciones anuales de los rendimientos diarios con SPX. 
Como una forma de hacer esto, primero creamos una función que calcula la correlación por pares de cada columna con la columna 'SPX':

In [0]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [0]:
rets = close_px.pct_change().dropna() #Calculamos el procentaje de cambio

In [0]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year) # Agrupamos los porcentajes de cambio por año
by_year.apply(spx_corr) # Llama a la funcion spx_corr para calcular la correlación de cada columna 

## Pivot Tables y tabulación cruzada

Una tabla dinámica es una herramienta de resumen de datos que se encuentra con frecuencia en programas de hojas de cálculo. 
Agrega una tabla de datos por una o más claves, organizando los datos en un rectángulo con algunas de las claves de grupo a lo largo de las filas y algunas a lo largo de las columnas. 
Las tablas dinámicas en Python con pandas son posibles a través de la función *groupby*. DataFrame tiene un método *pivot_table* y también hay una función *pandas.pivot_table* de nivel superior. Además de proporcionar una interfaz conveniente para *groupby*, *pivot_table* puede agregar totales parciales, también conocidos como márgenes.

Volviendo al conjunto de datos de propinas, supongamos que deseamos calcular una tabla de medios grupales

In [0]:
prop.pivot_table(index=['day', 'smoker'])

Ahora, supongamos que queremos aggregate solo *tip_pct* y *size*, y además agrupar por tiempo. Pondremos fumador en las columnas de la tabla y día en las filas:

In [0]:
prop.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

In [0]:
prop.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0) # Si hay NaN podemos usar fill_value

### Tabulaciones cruzadas (crosstab)
Una tabulación cruzada es un caso especial de una tabla dinámica que calcula las frecuencias de grupo. Aquí hay un ejemplo:

In [0]:
pd.crosstab([prop.time, prop.day], prop.smoker)

Podríamos aumentar esta tabla para incluir totales parciales pasando 'margins=true'. Esto tiene el efecto de agregar todas las etiquetas de fila y columna, siendo los valores correspondientes las estadísticas de grupo para todos los datos dentro de un solo nivel

In [0]:
pd.crosstab([prop.time, prop.day], prop.smoker, margins=True)